In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from statistics import median
import pickle
import csv
import multiprocess
import warnings
warnings.filterwarnings("ignore")

In [2]:
MAX_BATCH = [1, 2, 4, 8, 16]
algorithm = ['BATCHBISECT', 'BATCH4', 'BATCHSTOP4']
confidence = list(range(2,21,1))

In [3]:
projects = ['vagrant.csv', 'puppet.csv', 'graylog2-server.csv', 'heroku.csv', 'rails.csv', 'gradle.csv', 'jruby.csv', 'metasploit-framework.csv', 'cloudify.csv', 'rubinius.csv', 'open-build-service.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'opal.csv', 'cloud_controller_ng.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv']
data_path = '../data/exact_data/'

In [4]:
result_file = open('trial_aici_results.csv', 'w')
result_headers = ['version', 'project', 'algorithm', 'batch_size', 'project_reqd_builds', 'total_delay', 'failures_found', 'failures_not_found', 'bad_builds', 'batch_delays', 'testall_size', 'ci']
writer = csv.writer(result_file)
writer.writerow(result_headers)
result_file.close()

In [5]:
def get_pass_streak(y_project):
    p = y_project[0]
    pass_streak = [y_project[0]]
    for i in range(1, len(y_project)):
        pass_streak.append(p)
        if y_project[i] == 1:
            p += 1
        else:
            p = 0
    return pass_streak

In [6]:
def get_complete_data(p_name, first_failures=True):
    
    #open the metrics file
    filename = 'project_metrics/' + p_name.split('.')[0] + '_metrics.csv'
    project = pd.read_csv(filename)
    project = project.drop(project.columns[9], axis=1)
    project['tr_status'] = output_values(project['tr_status'])
    if first_failures:
        project = get_first_failures(project)
    return project

In [7]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [8]:
def hybrid_performance(p_name, test_builds, test_result, batchsize, ci):
    total_builds = len(test_result)

    bad_builds = 0
    flag = 0
    for i in range(len(test_result)):
        if flag == 1:
            if ci[i] == 1:
                bad_builds += 1
            else:
                flag == 0
        else:
            if test_result[i] == 0:
                if ci[i] == 1:
                    flag = 1
                    bad_builds += 1

    

    delay = []
    delay_indexes = []
    built_indexes = []
    for i in range(len(test_result)):
        if ci[i] == 0:
            built_indexes.append(i)
        if test_result[i] == 0:
            if ci[i] != 0:
                delay_indexes.append(i)
    
    num_failed = test_result.count(0)
    if num_failed == 0:
        failures_found = 100
        failures_not_found = 0

    else:
        num_of_failure_unidentified = len(delay_indexes)
        identified_failures = test_result.count(0) - num_of_failure_unidentified
        failures_found = 100*identified_failures/test_result.count(0)
        failures_not_found = 100*num_of_failure_unidentified/test_result.count(0)
    
    from_value = 0
    
    for k in range(len(built_indexes)):
        for j in range(len(delay_indexes)):
            if delay_indexes[j] > from_value and delay_indexes[j] < built_indexes[k]:
                delay.append(built_indexes[k] - delay_indexes[j])
        from_value = built_indexes[k]
    
    if len(delay_indexes) != 0:
        final_index = len(test_result)
        for j in range(len(delay_indexes)):
            delay.append(final_index - delay_indexes[j])
    
    return (sum(delay), failures_found, failures_not_found, bad_builds)

In [9]:
batch_total = 0
batch_durations = 0

In [10]:
def batch_bisect(batch_results):
    global batch_total
    
    batch_total += 1
    
    if len(batch_results) == 1:
        return
    
    if 0 in batch_results:
        half_batch = len(batch_results)//2
        batch_bisect(batch_results[:half_batch])
        batch_bisect(batch_results[half_batch:])

In [17]:
def batch_stop_4(batch_results):
    global batch_total
    
    batch_total += 1
    
    if len(batch_results) <= 4:
        if 0 in batch_results:
            batch_total += 4
        return
    
    if 0 in batch_results:
        half_batch = len(batch_results)//2
        batch_stop_4(batch_results[:half_batch])
        batch_stop_4(batch_results[half_batch:])

In [18]:
def get_predictions(p):
    
    test_file = "../data/exact_data/" + p + '.csv'
    project = pd_get_train_test_data(test_file, first_failures=False)
    
    
    #Received all predictions
    predictions = []
    Y_test = []
    X_test = pd.DataFrame()
    
    for ver in range(1, 11):

        model_file_name = '../../RQ2-Models/' + p + '_models/rq2_' + p + '_' + str(ver) + '_best_model.pkl'
        model_file = open(model_file_name, 'rb')
        predictor = pickle.load(model_file)
        threshold = pickle.load(model_file)
        

        x_test = project [ project['version'] == ver]
        if len(x_test) == 0:
            continue
        y_test = x_test['tr_status'].tolist()

        x_test.drop('tr_build_id', inplace=True, axis=1)
        x_test.drop('tr_status', inplace=True, axis=1)
        x_test.drop('version', inplace=True, axis=1)
        
        preds = predictor.predict(x_test)
        
        X_test = X_test.append(x_test)
        predictions.extend(preds)
        Y_test.extend(y_test)
    
    return predictions, Y_test, X_test

In [19]:
def mlci_process(p_name):
    
    global batch_total
    global batch_duration
    
    p = p_name.split('.')[0]
    
    print('Processing {}'.format(p_name))

    result_file = open('trial_aici_results.csv', 'a+')
    writer = csv.writer(result_file)
    
    project = get_complete_data(p_name, first_failures=False)
    
    prev = None
    for ver in range(1, 11):
        
        #get the test dataset indexes
        pkl_file = '../data/project_data_pickles/' + p_name + '_' + str(ver) + '_indexes.pkl'
        with open(pkl_file, 'rb') as load_file:
            train_build_ids = pickle.load(load_file)
            test_build_ids = pickle.load(load_file)
            
        
        #form the test dataset
        test_data = project [ project['tr_build_id'].isin(test_build_ids)]
        test_result = test_data['tr_status'].tolist()

        if len(test_result) == 0:
            continue 

        test_builds = test_data['tr_build_id'].tolist()
        test_data.drop('tr_build_id', inplace=True, axis=1)
        test_data.drop('tr_status', inplace=True, axis=1)
        test_data.drop('version', inplace=True, axis=1)


        file_name = '../../RQ3-Models/' + p + '_models/' + 'rq3_' + p_name + '_' + str(ver) + '_best_model.pkl'
        load_file = open(file_name, 'rb')
        forest = pickle.load(load_file)
        threshold = pickle.load(load_file)
    

        batchsizelist = [2, 4, 8, 16]
        algorithms = ['BATCH4', 'BATCHSTOP4', 'BATCHBISECT']

        batch_delays = 0
        final_pred_result = []

        for alg in algorithms:
            for batchsize in batchsizelist:

                batch_delays = 0
                
                if prev == None:
                    pass_streak = 0
                else:
                    pass_streak = prev
                i = 0
                total = len(test_data)
                num_of_builds = 0
                build_duration = 0

                #The variable 'ci' will hold the actual execution process of the current phase
                #If ci[i] == 0, it means that build was made
                #If ci[i] == 1, it means that build was saved
                ci = []

                if alg == 'NOBATCH':
                    while i < total:
                        data = test_data.iloc[i]
                        data['num_of_passes'] = pass_streak
                        predict = grid_search.predict_proba([data])

                        if predict[0][1] > threshold:
                            final_pred_result.append(1)
                            ci.append(1)
                            pass_streak += 1
                            i += 1
                        else:

                            while i < total:
                                if (total - i) > batchsize:
                                    ci.extend([0 for i in range(batchsize)])
                                else:
                                    ci.extend([0 for e in range(total-i)])

                                batch_delays += (batchsize - 1)*batchsize*0.5

                                batch_build_times = durations[i:i+4]
                                actual_batch_results = test_result[i:i+4]

                                num_of_builds += 1
                                build_duration += max(batch_build_times)

                                if 0 in actual_batch_results:
                                    i = i+batchsize
                                else:
                                    break
                            i += batchsize
                            pass_streak = 1


                if alg == 'BATCH4':
                    if batchsize != 4:
                        continue
                    else:
                            while i < total :
                                data = test_data.iloc[i]
                                data['num_of_passes'] = pass_streak
                                predict = forest.predict_proba([data])

                                #predicted that build has passed
                                if predict[0][1] > threshold:
                                    final_pred_result.append(1)
                                    ci.append(1)
                                    pass_streak += 1
                                    i+=1

                                else:
                                    #We found first failure

                                    #Until an entire batch passes, we are going to continue group builds ie., subsequent failures are grouped
                                    while i < total:
                                        if (total - i) > 4:
                                            ci.extend([0,0,0,0])
                                        else:
                                            ci.extend([0 for e in range(total-i)])

                                        batch_delays += (batchsize - 1)*batchsize*0.5

                                        actual_batch_results = test_result[i:i+4]

                                        num_of_builds += 1                                    
                                            # print(batch_build_times)
                                            # print(durations)
                                            # print(test_result)

                                        #if any build has failed in the batch, then whole batch will fail
                                        if 0 in actual_batch_results:
                                            i = i+4
                                            num_of_builds += 4
                                        else:
                                            break
                                    #Now that we have found a passing build, we can update pass_streak to 1
                                    pass_streak = 1
                                    i += 4

                if alg == 'BATCHSTOP4':
                    if batchsize < 4:
                        continue
                    else:
                        pass_streak = 0
                        ci = []
                        while i < total :
                            data = test_data.iloc[i]
                            data['num_of_passes'] = pass_streak
                            predict = forest.predict_proba([data])

                            if predict[0][1] > threshold:
                                ci.append(1)
                                pass_streak += 1
                                i += 1
                            else:

                                while i < total:
                                    if (total - i) > batchsize:
                                        ci.extend([0 for l in range(batchsize)])
                                    else:
                                        ci.extend([0 for e in range(total-i)])


                                    batch_delays += (batchsize - 1)*batchsize*0.5

                                    grouped_batch_results = test_result[i:i+batchsize]
                                    batch_total = 0

                                    batch_stop_4(grouped_batch_results)
                                    num_of_builds += batch_total

                                    if 0 not in grouped_batch_results:
                                        break
                                    else:
                                        i += batchsize
                                    grouped_batch_results.clear()
                                i += batchsize
                                pass_streak = 1

                if alg == 'BATCHBISECT':

                    pass_streak = 0
                    ci = []

                    while i < total :
                        data = test_data.iloc[i]
                        data['num_of_passes'] = pass_streak
                        predict = forest.predict_proba([data])

                        if predict[0][1] > threshold:
                            ci.append(1)
                            pass_streak += 1
                            i += 1
                        else:

                            #this case is when model has predicted a failure
                            #Add [i, i+batchsize] to a group and perform BatchBisect

                            while i < total:

                                #Next batch is being built, so add to ci
                                if (total - i) >= batchsize:
                                    ci.extend([0 for l in range(batchsize)])
                                else:
                                    ci.extend([0 for e in range(total-i)])

                                batch_delays += (batchsize - 1)*batchsize*0.5

                                grouped_batch_results = test_result[i:i+batchsize]

                                batch_total = 0

                                batch_bisect(grouped_batch_results)
                                num_of_builds += batch_total

                                if 0 not in grouped_batch_results:
                                    break
                                else:
                                    i += batchsize

                                grouped_batch_results.clear()
                            i += batchsize
                            pass_streak = 1


                batch_performance = hybrid_performance(p_name, test_builds, test_result, batchsize, ci)
                total_delay = batch_performance[0]
                failures_found = batch_performance[1]
                failures_not_found = batch_performance[2]
                bad_builds = batch_performance[3]

                local_builds_reqd = 100*num_of_builds/total

                print([ver, p_name, alg, batchsize, local_builds_reqd, total_delay, failures_found, failures_not_found, bad_builds, batch_delays, total, ci])
                writer.writerow([ver, p_name, alg, batchsize, local_builds_reqd, total_delay, failures_found, failures_not_found, bad_builds, batch_delays, total, ci])
                if total != len(ci):
                    print('PROBLEM!!')
                else:
                    print('NO PROBLEM!!')
                
                pass_streak = ci[-1]


In [20]:
for pr in projects:
    mlci_process(pr)

Processing vagrant.csv
[3, 'vagrant.csv', 'BATCH4', 4, 29.526462395543174, 687, 71.42857142857143, 28.571428571428573, 17, 516.0, 359, [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[4, 'vagrant.csv', 'BATCH4', 4, 23.076923076923077, 227, 90.9090909090909, 9.090909090909092, 58, 756.0, 702, [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0

[4, 'vagrant.csv', 'BATCHBISECT', 2, 30.34188034188034, 1735, 63.63636363636363, 36.36363636363637, 276, 199.0, 702, [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 

[5, 'vagrant.csv', 'BATCH4', 4, 39.96101364522417, 0, 100.0, 0.0, 0, 1500.0, 1026, [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[5, 'vagrant.csv', 'BATCHSTOP4', 16, 32.748538011695906, 0, 100.0, 0.0, 0, 7680.0, 1026, [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

[4, 'orbeon-forms.csv', 'BATCHSTOP4', 8, 70.29702970297029, 112, 98.82352941176471, 1.1764705882352942, 2, 672.0, 202, [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
NO PROBLEM!!
[4, 'orbeon-forms.csv', 'BATCHSTOP4', 16, 79.70297029702971, 0, 100.0, 0.0, 0, 1560.0, 202, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

[5, 'orbeon-forms.csv', 'BATCHBISECT', 4, 107.66129032258064, 0, 100.0, 0.0, 0, 366.0, 248, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
NO PROBLEM!!
[5, 'orbeon-forms.csv', 'BATCHBISECT', 8, 115.7258064516129, 0, 100.0, 0.0, 0, 868.0, 248, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[7, 'orbeon-forms.csv', 'BATCHSTOP4', 16, 139.41176470588235, 339, 98.70967741935483, 1.2903225806451613, 3, 1320.0, 170, [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
NO PROBLEM!!
[7, 'orbeon-forms.csv', 'BATCHBISECT', 2, 138.8235294117647, 804, 96.7741935483871, 3.225806451612903, 7, 82.0, 170, [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[9, 'orbeon-forms.csv', 'BATCH4', 4, 99.0, 0, 100.0, 0.0, 0, 300.0, 200, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
NO PROBLEM!!
[9, 'orbeon-forms.csv', 'BATCHSTOP4', 4, 99.0, 0, 100.0, 0.0, 0, 300.0, 200, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[10, 'orbeon-forms.csv', 'BATCHBISECT', 2, 72.5, 0, 100.0, 0.0, 0, 116.0, 240, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
NO PROBLEM!!
[10, 'orbeon-forms.csv', 'BATCHBISECT', 4, 71.25, 0, 100.0, 0.0, 0, 354.0, 240, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 